In [1]:
# i am importing numpy and pands for working with data
import numpy as np
import pandas as pd

# for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# for preprocessing
from pathlib import Path
from sklearn.model_selection import train_test_split

# i'll get my model up and running with tensorflow
import tensorflow as tf

# evaluating the performance using sklearn
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# using flow from data frame
positive_dir = Path('../input/surface-crack-detection/Positive')
negative_dir = Path('../input/surface-crack-detection/Negative')

In [3]:
# creating the data frame
def generate_df(image_dir, label):
    filepaths = pd.Series(list(image_dir.glob(r'*.jpg')), name='Filepath').astype(str)
    labels = pd.Series(label, name='Label', index=filepaths.index)
    df = pd.concat([filepaths, labels], axis=1)
    return df

In [4]:
positive_df = generate_df(positive_dir, label="POSITIVE")
negative_df = generate_df(negative_dir, label="NEGATIVE")

all_df = pd.concat([positive_df, negative_df], axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
print(all_df)

                                                Filepath     Label
0      ../input/surface-crack-detection/Positive/0574...  POSITIVE
1      ../input/surface-crack-detection/Positive/1870...  POSITIVE
2      ../input/surface-crack-detection/Positive/0967...  POSITIVE
3      ../input/surface-crack-detection/Negative/0791...  NEGATIVE
4      ../input/surface-crack-detection/Positive/1400...  POSITIVE
...                                                  ...       ...
39995  ../input/surface-crack-detection/Positive/0854...  POSITIVE
39996  ../input/surface-crack-detection/Negative/1944...  NEGATIVE
39997  ../input/surface-crack-detection/Positive/0977...  POSITIVE
39998  ../input/surface-crack-detection/Positive/1504...  POSITIVE
39999  ../input/surface-crack-detection/Negative/1099...  NEGATIVE

[40000 rows x 2 columns]


In [5]:
# doing the train test split from sklearn
train_df, test_df = train_test_split(
    all_df.sample(6000, random_state=1),
    train_size=0.7,
    shuffle=True,
    random_state=1
)

In [6]:
train_df

,Filepath,Label
30189,../input/surface-crack-detection/Negative/1208...,NEGATIVE
8931,../input/surface-crack-detection/Negative/0385...,NEGATIVE
29084,../input/surface-crack-detection/Positive/0641...,POSITIVE
29189,../input/surface-crack-detection/Negative/1611...,NEGATIVE
2645,../input/surface-crack-detection/Negative/1833...,NEGATIVE
...,...,...
2090,../input/surface-crack-detection/Positive/0836...,POSITIVE
35101,../input/surface-crack-detection/Positive/1839...,POSITIVE
8720,../input/surface-crack-detection/Positive/0939...,POSITIVE
9955,../input/surface-crack-detection/Positive/0140...,POSITIVE


In [7]:
test_df

,Filepath,Label
15731,../input/surface-crack-detection/Negative/0694...,NEGATIVE
34272,../input/surface-crack-detection/Negative/0744...,NEGATIVE
39532,../input/surface-crack-detection/Positive/0052...,POSITIVE
10818,../input/surface-crack-detection/Positive/0208...,POSITIVE
35421,../input/surface-crack-detection/Negative/0618...,NEGATIVE
...,...,...
7174,../input/surface-crack-detection/Positive/1625...,POSITIVE
27207,../input/surface-crack-detection/Negative/0607...,NEGATIVE
28566,../input/surface-crack-detection/Negative/1254...,NEGATIVE
20688,../input/surface-crack-detection/Positive/0636...,POSITIVE


In [8]:
# loading the image data
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    # rescaling it because I want to make sure that the actual image data is comming between values 0 and 1
    rescale=1./255,
    validation_split=0.2
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [9]:
# flowing data from the data frames
train_data = train_gen.flow_from_dataframe(
    train_df,
    # specifying which column is used as a filepath and which column is used as labels
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_data = train_gen.flow_from_dataframe(
    test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False,
    seed=42
)

Found 3360 validated image filenames belonging to 2 classes.
Found 840 validated image filenames belonging to 2 classes.
Found 1800 validated image filenames belonging to 2 classes.
